# Drivers for Assignment 5 - CPSC 302 - Winter 1 - 2024

In [ ]:
function StationaryMethod(A, b, M, itermax, tol, x)
    # Apply a stationary method to a linear system
    #
    # Input:
    # A - matrix
    # b - right-hand side vector
    # M - matrix associated with splitting (determines the method)
    # itermax - maximum allowed number of iterations
    # tol - convergence threshold: terminate if relative residual norm goes below
    # x - initial guess
    #
    # Output:
    # x - numerical solution (initial guess is overwritten)
    # i_max - iteration count
    # t - CPU time taken (in seconds)
    # resvec - residual norm convergence history

    resvec = zeros(itermax) # Allocate space for convergence history
    nb = norm(b - A * x)    # Initial residual
    t_start = time()        # Record start time
    i_max=itermax
    for i in 1:itermax
        r = b - A * x                # Update residual
        resvec[i] = norm(r) / nb     # Relative residual norm

        # Terminate loop if stopping criterion is satisfied:
        if resvec[i] < tol
            resvec = resvec[1:i]     # Trim unused space in `resvec`
            i_max=i
            break
        end

        x = x + M \ r                # Next iterate
    end

    t_total = time() - t_start       # Compute total CPU time

    return x, i_max, t_total, resvec
end


In [ ]:
# Script: Apply the Jacobi and Gauss-Seidel methods to a linear system

using LinearAlgebra, SparseArrays
using Plots  # Ensure the Plots.jl package is installed

# Problem setup
n = 100
Δ=spdiagm(-1=> -1*ones(n-1),0=> 2*ones(n),1=> -1*ones(n-1))
D_xx = kron(Δ, sparse(I,n,n))
D_yy = kron(sparse(I,n,n),Δ)
A = D_xx + D_yy
n2 = size(A, 1)
b = A * ones(n2)               # Right-hand side vector
x = zeros(n2)                  # Initial guess
itermax = 100000               # Maximum iterations
tol = 1e-8                     # Convergence tolerance

# Jacobi iteration
D = Diagonal(diagm(0 => diag(A)))  # M = D for Jacobi
xJ, iterJac, tJ, resvecJ = StationaryMethod(A, b, D, itermax, tol, x)
println("CPU time for Jacobi: $tJ seconds")
println("Iteration count for Jacobi: $iterJac iterations")

# Gauss-Seidel iteration
E = LowerTriangular(A)         # M = E for Gauss-Seidel
xGS, iterGS, tGS, resvecGS = StationaryMethod(A, b, E, itermax, tol, x)
println("CPU time for Gauss-Seidel: $tGS seconds")
println("Iteration count for Gauss-Seidel: $iterGS iterations")

# Plot convergence history
plot(1:length(resvecJ), resvecJ, yscale=:log10, label="Jacobi", xlabel="Iterations", ylabel="Relative residual norm", lw=2)
plot!(1:length(resvecGS), resvecGS, label="Gauss-Seidel", color=:red, lw=2)
